In [80]:
import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.lines as mlines

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from scipy import stats
from scipy import linalg
from scipy.stats import skew
from scipy.stats import kurtosis
from scipy.stats import entropy
import statsmodels.api as sm
from statsmodels.formula.api import ols
import statsmodels.stats.multicomp as multi
from scipy.stats import levene
from scipy.stats import shapiro
from scipy.integrate import cumtrapz
from scipy.linalg import eig
from numpy.fft import fft


#from sklearn.cluster import KMeans
#from sklearn.preprocessing import StandardScaler # z = (x - u) / s

from copy import deepcopy
import warnings

In [81]:
# excercise 2

def dataset(patient_id):
    head=["device_id","acc_x","acc_y","acc_z","gyro_x","gyro_y","gyro_z","mag_x","mag_y","mag_z","time","activity","patient_id"]

    #device_1 = pd.read_csv("dataset/part{patient}/part{patient}dev1.csv".format(patient=patient_id),names=head)
    device_2 = pd.read_csv("dataset/part{patient}/part{patient}dev2.csv".format(patient=patient_id),names=head)
    #device_3 = pd.read_csv("dataset/part{patient}/part{patient}dev3.csv".format(patient=patient_id),names=head)
    #device_4 = pd.read_csv("dataset/part{patient}/part{patient}dev4.csv".format(patient=patient_id),names=head)
    #device_5 = pd.read_csv("dataset/part{patient}/part{patient}dev5.csv".format(patient=patient_id),names=head)
    
    
    #df=pd.concat([device_1,device_2,device_3,device_4,device_5],axis=0)
    df=device_2
    df["patient_id"]=patient_id
    return df

In [5]:
# excercise 2

def transformed_dataset():

    df_transformed = pd.DataFrame()

    for i in range (0,15):
        df_transformed = df_transformed.append(dataset(i))
    df_transformed = df_transformed.reset_index()
        
    df_transformed["acc_vector"] = df_transformed["acc_x"]**2+df_transformed["acc_y"]**2+df_transformed["acc_z"]**2
    df_transformed["gyro_vector"] = df_transformed["gyro_x"]**2+df_transformed["gyro_y"]**2+df_transformed["gyro_z"]**2
    df_transformed["mag_vector"] = df_transformed["mag_x"]**2+df_transformed["mag_y"]**2+df_transformed["mag_z"]**2
    
    return df_transformed[["device_id","acc_vector","gyro_vector","mag_vector","time","activity","patient_id"]]

In [6]:
# excercise 3.1

def vector_boxplot(sensor_id):
    
    df_transformed = transformed_dataset()
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 10))
    sns.set_style("darkgrid") 
    
    if sensor_id=="accelerometer":
        ax=sns.boxplot(data=df_transformed, y="acc_vector",x="activity", palette="spring")
      
    elif sensor_id=="gyroscope":
        ax=sns.boxplot(data=df_transformed, y="gyro_vector",x="activity", palette="spring")
                
    elif sensor_id=="magnetometer":
        ax=sns.boxplot(data=df_transformed, y="mag_vector",x="activity", palette="spring")
        
    else:
        return "enter a correct name"   

    ax.set_title("Norm from {sensor}".format(sensor=sensor_id), size=15)  
    ax.set_xlabel("Activities",size=15)
    
    x_labels = np.arange(0,16)
    x_new_labels = ["1","2","3","4","5","6","7","8","9","10","11","12","13","14","15","16"]
    
    ax.set_xticklabels(labels=np.arange(0,16),size=12)
    ax.set_ylabel("")
    plt.xticks(x_labels, x_new_labels, size=12)
    plt.savefig("Norm from {sensor}".format(sensor=sensor_id))
    
    return plt.show()   

In [7]:
# excercise 3.2 by activity

def density_by_activity(k, act):
    
    density_by_activity=pd.DataFrame(columns=["density_acc","density_gyro","density_mag"])
    density_by_activity.index.name = "Activities"
    
    for act in range(1,17):
        df_transformed = transformed_dataset()
        df_transformed = df_transformed[df_transformed["activity"]==act].reset_index(drop=True)
        
        mean = df_transformed[["acc_vector","gyro_vector","mag_vector"]].mean()
        std = df_transformed[["acc_vector","gyro_vector","mag_vector"]].std()
    
        df_transformed["outlier_acc"] = (df_transformed["acc_vector"] < (mean["acc_vector"]-k*std["acc_vector"])) | (df_transformed["acc_vector"] > (mean["acc_vector"]+k*std["acc_vector"]))
        df_transformed["outlier_gyro"] = (df_transformed["gyro_vector"] < (mean["gyro_vector"]-k*std["gyro_vector"])) | (df_transformed["gyro_vector"] > (mean["gyro_vector"]+k*std["gyro_vector"]))
        df_transformed["outlier_mag"] = (df_transformed["mag_vector"] < (mean["mag_vector"]-k*std["mag_vector"])) | (df_transformed["mag_vector"] > (mean["mag_vector"]+k*std["mag_vector"]))
        density_acc = df_transformed["outlier_acc"].sum() / df_transformed["acc_vector"].count() * 100
        density_gyro = df_transformed["outlier_gyro"].sum() / df_transformed["gyro_vector"].count() * 100
        density_mag = df_transformed["outlier_mag"].sum() / df_transformed["mag_vector"].count() * 100
        density_by_activity.loc[act]=[density_acc,density_gyro,density_mag]

    return density_by_activity

In [8]:
# excercise 3.2

def density(k, act):
    
    df_transformed = transformed_dataset()
    df_transformed = df_transformed[df_transformed["activity"]==act].reset_index(drop=True)
    mean = df_transformed[["acc_vector","gyro_vector","mag_vector"]].mean()
    std = df_transformed[["acc_vector","gyro_vector","mag_vector"]].std()
    
    df_transformed["outlier_acc"] = (df_transformed["acc_vector"] < (mean["acc_vector"]-k*std["acc_vector"])) | (df_transformed["acc_vector"] > (mean["acc_vector"]+k*std["acc_vector"]))
    df_transformed["outlier_gyro"] = (df_transformed["gyro_vector"] < (mean["gyro_vector"]-k*std["gyro_vector"])) | (df_transformed["gyro_vector"] > (mean["gyro_vector"]+k*std["gyro_vector"]))
    df_transformed["outlier_mag"] = (df_transformed["mag_vector"] < (mean["mag_vector"]-k*std["mag_vector"])) | (df_transformed["mag_vector"] > (mean["mag_vector"]+k*std["mag_vector"]))
    density_acc = df_transformed["outlier_acc"].value_counts()[True] / df_transformed["acc_vector"].count() * 100
    density_gyro = df_transformed["outlier_gyro"].value_counts()[True] / df_transformed["gyro_vector"].count() * 100
    density_mag = df_transformed["outlier_mag"].value_counts()[True] / df_transformed["mag_vector"].count() * 100
    
    z_score_outliers=pd.DataFrame(df_transformed[["outlier_acc","outlier_gyro","outlier_mag"]])
    
    return density_acc, density_gyro, density_mag, z_score_outliers

In [9]:
# excercise 3.2

def density_activity(k,act):
     
    df_transformed = transformed_dataset()
    df_activity = df_transformed[df_transformed["activity"]==act].copy()
    mean = df_activity[["acc_vector","gyro_vector","mag_vector"]].mean()
    std = df_activity[["acc_vector","gyro_vector","mag_vector"]].std()
    
    df_activity["outlier_acc"] = (df_activity["acc_vector"] < (mean["acc_vector"]-k*std["acc_vector"])) | (df_activity["acc_vector"] > (mean["acc_vector"]+k*std["acc_vector"]))
    df_activity["outlier_gyro"] = (df_activity["gyro_vector"] < (mean["gyro_vector"]-k*std["gyro_vector"])) | (df_activity["gyro_vector"] > (mean["gyro_vector"]+k*std["gyro_vector"]))
    df_activity["outlier_mag"] = (df_activity["mag_vector"] < (mean["mag_vector"]-k*std["mag_vector"])) | (df_activity["mag_vector"] > (mean["mag_vector"]+k*std["mag_vector"]))
#    density_acc = df_activity["outlier_acc"].value_counts()[True] / df_activity["acc_vector"].count() * 100
#    density_gyro = df_activity["outlier_gyro"].value_counts()[True] / df_activity["gyro_vector"].count() * 100
#    density_mag = df_activity["outlier_mag"].value_counts()[True] / df_activity["mag_vector"].count() * 100
    density_acc = df_activity[df_activity["outlier_acc"]==True].count()["outlier_acc"] / df_activity["acc_vector"].count() * 100
    density_gyro = df_activity[df_activity["outlier_gyro"]==True].count()["outlier_gyro"] / df_activity["gyro_vector"].count() * 100
    density_mag = df_activity[df_activity["outlier_mag"]==True].count()["outlier_mag"] / df_activity["mag_vector"].count() * 100
    
    return density_acc, density_gyro, density_mag, df_activity

In [10]:
# excercise 3.3

def outlier_Zscore_by_activity(k,act):
    warnings.filterwarnings("ignore")
    
    df_transformed = transformed_dataset()
    df_transformed = df_transformed[df_transformed["activity"]==act].reset_index(drop=True)
    mean = df_transformed[["acc_vector","gyro_vector","mag_vector"]].mean()
    std = df_transformed[["acc_vector","gyro_vector","mag_vector"]].std()
    
    Z = (df_transformed[["acc_vector","gyro_vector","mag_vector"]]-mean)/std
    outliers = abs(Z)>k
    
    non_acc_outliers = df_transformed[outliers["acc_vector"]==False]["acc_vector"]
    non_gyro_outliers = df_transformed[outliers["gyro_vector"]==False]["gyro_vector"]
    non_mag_outliers = df_transformed[outliers["mag_vector"]==False]["mag_vector"]
    
    df_describe_non_outliers=pd.DataFrame({"acc_vector":non_acc_outliers,}).describe().loc[["min","max","mean","std","count"]]
    df_describe_non_outliers["gyro_vector"]=pd.DataFrame({"gyro_vector":non_gyro_outliers,}).describe().loc[["min","max","mean","std","count"]]
    df_describe_non_outliers["mag_vctor"]=pd.DataFrame({"mag_vector":non_mag_outliers,}).describe().loc[["min","max","mean","std","count"]]
    
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(20, 10))

    sns.distplot(df_transformed["acc_vector"], ax=ax[0], bins=100, label="with_outliers")
    sns.distplot(non_acc_outliers, ax=ax[0], color="red", bins=100, label="without_outliers")
    ax[0].set_xlim(95,110)
    ax[0].set_ylim(0, 0.25)
    ax[0].set_title("Dataset distribution for acc_vector for activity {}".format(act), size=14)
    ax[0].set_xlabel("acc_vector", size=14) 
    ax[0].set_ylabel("")
    ax[0].legend(prop=dict(size=12))

    sns.distplot(df_transformed["gyro_vector"], ax=ax[1], bins=100, label="with_outliers")
    sns.distplot(non_gyro_outliers, ax=ax[1], color="red", bins=100, label="without_outliers")
    ax[1].set_xlim(0,180)
    ax[1].set_ylim(0, 0.01)
    ax[1].set_title("Dataset distribution for gyro_vector for activity {}".format(act), size=14)
    ax[1].set_xlabel("gyro_vector", size=14)
    ax[1].set_ylabel("")
    ax[1].legend(prop=dict(size=12))

    sns.distplot(df_transformed["mag_vector"], ax=ax[2], bins=100, label="with_outliers")
    sns.distplot(non_mag_outliers, ax=ax[2], color="red", bins=100, label="without_outliers")
    ax[2].set_ylim(0, 1)
    ax[2].set_xlim(1.5,3.5)
    ax[2].set_title("Dataset distribution for mag_vector for activity {}".format(act), size=14)
    ax[2].set_ylabel("")
    ax[2].set_xlabel("mag_vector", size=14)
    ax[2].legend(prop=dict(size=12))

    plt.savefig("Dataset distribution")
    plt.show()
    
    return df_describe_non_outliers, df_transformed[outliers["acc_vector"]==False][["acc_vector","time"]]

In [11]:
# excercise 3.3

def outlier_Zscore(amostra,k):
    
    mean = amostra.mean()
    std = amostra.std()
    Z = (amostra-mean)/std
    outliers = abs(Z)>k
    
    return outliers

In [12]:
# excercise 3.4

#https://stackoverflow.com/questions/29779079/adding-a-scatter-of-points-to-a-boxplot-using-matplotlib

def plot_outlier(k,sensor_id):
    
    df_transformed = transformed_dataset()
    
    plt.rcParams["figure.figsize"] = (10,10)
    boxprops = dict(linewidth=1.0) 
    

    if sensor_id=="accelerometer":         
        df_transformed["outlier_acc_Zscore"] = outlier_Zscore(df_transformed["acc_vector"],k)        
        df_transformed.boxplot(column='acc_vector', by='activity', grid=True, showfliers=False, boxprops = boxprops, patch_artist=True)
        
        for i in range(0,17):
            
            y = df_transformed["acc_vector"][(df_transformed["activity"]==i) & (df_transformed["outlier_acc_Zscore"] == True)]
            x = np.random.normal(i, 0.01, size=len(y))
            plt.plot(x, y, 'ro',markersize = 4, zorder=2)
            
            y = df_transformed["acc_vector"][(df_transformed["activity"]==i) & (df_transformed["outlier_acc_Zscore"] == False)]
            x = np.random.normal(i, 0.01, size=len(y))
            plt.plot(x, y, 'bo', markersize = 4, zorder=1)
             
    elif sensor_id=="gyroscope":
        df_transformed["outlier_gyro_Zscore"] = outlier_Zscore(df_transformed["gyro_vector"],k)        
        df_transformed.boxplot(column='gyro_vector', by='activity', grid=True, showfliers=False, boxprops = boxprops, patch_artist=True)
        
        for i in range(0,17):           
            y = df_transformed["gyro_vector"][(df_transformed["activity"]==i) & (df_transformed["outlier_gyro_Zscore"] == True)]
            x = np.random.normal(i, 0.01, size=len(y))
            plt.plot(x, y, 'ro', markersize = 4, zorder=2)
            
            y = df_transformed.gyro_vector[(df_transformed["activity"]==i) & (df_transformed["outlier_gyro_Zscore"] == False)]
            x = np.random.normal(i, 0.01, size=len(y))
            plt.plot(x, y, 'bo', markersize = 4, zorder=1)
                
    elif sensor_id=="magnetometer":
        
        df_transformed["outlier_mag_Zscore"] = outlier_Zscore(df_transformed["mag_vector"],k)        
        df_transformed.boxplot(column='mag_vector', by='activity', grid=True, showfliers=False, boxprops = boxprops, patch_artist=True)

        for i in range(0,17):
            y = df_transformed["mag_vector"][(df_transformed["activity"]==i) & (df_transformed["outlier_mag_Zscore"] == True)]
            x = np.random.normal(i, 0.01, size=len(y))
            plt.plot(x, y, 'ro', markersize = 4, zorder=2)
            
            y = df_transformed.mag_vector[(df_transformed["activity"]==i) & (df_transformed["outlier_mag_Zscore"] == False)]
            x = np.random.normal(i, 0.01, size=len(y))
            plt.plot(x, y, 'bo', markersize = 4, zorder=1)
        
    else:
        return "enter a correct name"  
    
    plt.title("Zscore outliers from {sensor}".format(sensor=sensor_id), size=15)  
    plt.xlabel("Activities",size=15)
    plt.legend(labels=["outliers"], loc="upper right", fontsize=12, handles = plt.plot([], marker="o" ,ls="", color="r"))
    plt.savefig("Zscore outliers from {sensor}".format(sensor=sensor_id))
 

In [13]:
#X=df_transformed[["acc_vector","gyro_vector","mag_vector"]]

In [14]:
# normalize data
#Zscore_scaler = StandardScaler()
#Zscore_scaler.fit(X)
#X_scaled=pd.DataFrame(Zscore_scaler.transform(X), columns=["acc_vector","gyro_vector","mag_vector"])

# denormalize data
#X = pd.DataFrame(Zscore_scaler.inverse_transform(X_scaled), columns=["acc_vector","gyro_vector","mag_vector"])

In [15]:
# exercise 3.6

#https://www.kaggle.com/andyxie/k-means-clustering-implementation-in-python

def KMeans(k, max_iterations,act):
    
    data = transformed_dataset()
    data = data[data["activity"]==act][["acc_vector","gyro_vector","mag_vector"]].reset_index(drop=True)

# Number of training data
    n = data.shape[0]
# Number of features in the data
    c = data.shape[1]

# Generate random centers, here we use sigma and mean to ensure it represent the whole data
    mean = np.mean(data, axis = 0).values
    std = np.std(data, axis = 0).values
    #centers = np.random.randn(k,c)*std + mean
    centers= data.sample(n = k).values

    centers_old = np.zeros(centers.shape) # to store old centers
    centers_new = deepcopy(centers) # Store new centers

    data.shape
    clusters_label = np.zeros(n)
    distances = np.zeros((n,k))

    error = np.linalg.norm(centers_new - centers_old)

# When, after an update, the estimate of that center stays the same, exit loop
    j=0
    while not (error < 0.001 or j == max_iterations):
        # Measure the distance to every center
        for i in range(k):
            distances[:,i] = np.linalg.norm(data - centers_new[i], axis=1)
        # Assign all data to closest center
        clusters_label = np.argmin(distances, axis = 1)
    
        centers_old = deepcopy(centers_new)
        # Calculate mean for every cluster and update the center
        for i in range(k):
            centers_new[i] = np.mean(data[clusters_label == i], axis=0)                
        error = np.linalg.norm(centers_new - centers_old)
        j += 1

    inertia = np.sum(np.min(distances, axis = 1))
   
    return centers_new, clusters_label, inertia

In [16]:
# exercise 3.6

def KMeans_plot(clusters_label, act):
    
    df_transformed = transformed_dataset()
    df_transformed = df_transformed[df_transformed["activity"]==act].reset_index(drop=True)
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1, 1, 1, projection="3d")

    p = ax.scatter(df_transformed["acc_vector"], df_transformed["gyro_vector"], df_transformed["mag_vector"], 
                   c=clusters_label, cmap="rainbow", zdir="-z")

    ax.set_title("K-Means clusters for activity {act} with k = 5".format(act=act), size=15)
    ax.set_xlabel("acc_vector", size=15)
    ax.set_ylabel("gyro_vector", size=15)
    ax.set_zlabel("mag_vector", size=15)
    plt.savefig("K-means clusters for activity {act} with k = 5".format(act=act))
    return plt.show()

In [17]:
# exercise 3.7

def elbow_method(max_clusters, max_it, act):
    
    df_transformed = transformed_dataset()
    df_transformed = df_transformed[df_transformed["activity"]==act][["acc_vector","gyro_vector","mag_vector"]].reset_index(drop=True)
    
    plt.figure(figsize=(12,8))
    seq = {}

    for i in range(1,max_clusters+1):
        _,_,inertia = KMeans(k=i, max_iterations=max_it,act=act)
        seq[i] = inertia
 
    plt.plot(list(seq.keys()),list(seq.values()))
    
    plt.title("k-means elbow method for activity {}".format(act), size=15) 
    plt.xlabel('Nº of clusters', size=15)
    plt.ylabel('Sum of squared distances to the center of cluster', size=15)
    plt.savefig("K-means elbow method for activity {}".format(act))
    return plt.show()

In [18]:
# exercise 3.7

def KMeans_outliers(density, centers_new, clusters_label, act):
     
    df_transformed = transformed_dataset()
    df_transformed = df_transformed[df_transformed["activity"]==act][["acc_vector","gyro_vector","mag_vector"]].reset_index(drop=True)
    
    center = centers_new
    df_transformed["kmeans_label"]=clusters_label
    
    center_points = np.zeros((len(df_transformed),3))
    for i in range(0,len(df_transformed)):
        label = df_transformed["kmeans_label"][i]
        center_points[i][:] = center[label][:]
        
    df_transformed["kmeans_distance"] = np.sqrt((df_transformed["acc_vector"]-center_points[:,0])**2 + 
                                         (df_transformed["gyro_vector"]-center_points[:,1])**2 + 
                                         (df_transformed["mag_vector"]-center_points[:,2])**2)
    max_distance = df_transformed.groupby(['kmeans_label'])['kmeans_distance'].max()
    max_points = np.zeros((len(df_transformed)))
    
    for i in range(0,len(df_transformed)):
        label = df_transformed["kmeans_label"][i]
        max_points[i] = max_distance[label]
        
    df_transformed["kmeans_outliers"] = ((df_transformed["kmeans_distance"]/max_points) > density) 
    
    x=df_transformed["acc_vector"][df_transformed["kmeans_outliers"] == False]
    y=df_transformed["gyro_vector"][df_transformed["kmeans_outliers"] == False]
    z=df_transformed["mag_vector"][df_transformed["kmeans_outliers"] == False]

    x_outlier=df_transformed["acc_vector"][df_transformed["kmeans_outliers"] == True]
    y_outlier=df_transformed["gyro_vector"][df_transformed["kmeans_outliers"] == True]
    z_outlier=df_transformed["mag_vector"][df_transformed["kmeans_outliers"] == True]
    
    colors=df_transformed["kmeans_label"][df_transformed["kmeans_outliers"] == False]
    
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(1,1,1, projection='3d')

    ax.scatter(x, y, z, cmap="rainbow", zdir="z",zorder=0, c=colors, alpha=0.6)
    ax.scatter(x_outlier, y_outlier, z_outlier, c="black", marker="p", zdir="z", zorder=20, label="outliers")
    
    ax.set_title("K-Means outliers", size=15)
    ax.set_xlabel("acc_vector", size=15)
    ax.set_ylabel("gyro_vector", size=15)
    ax.set_zlabel("mag_vector", size=15)
    ax.legend()
    plt.savefig("K-means outliers")

    plt.show()
    
    k_means_non_outliers=pd.DataFrame({"acc_vector":x,"gyro_vector":y,"mag_vector":z}).describe().loc[["min","max","mean","std","count"],["acc_vector","gyro_vector","mag_vector"]]
    return k_means_non_outliers

In [19]:
# exercise 3.8

def outliers_injection(x,k,act):
    
    df_transformed = transformed_dataset()
    df_transformed = df_transformed[df_transformed["activity"]==act].reset_index(drop=True)
    
    density_acc, density_gyro, density_mag,z_score_outliers = density(k, act)
    
    df_transformed = df_transformed.join(z_score_outliers)
    
    mean = df_transformed[["acc_vector","gyro_vector","mag_vector"]].mean()
    std = df_transformed[["acc_vector","gyro_vector","mag_vector"]].std()
    s = np.random.uniform(-1,1)
    
    range_acc = [abs(min(df_transformed["acc_vector"]) - (mean["acc_vector"]-k*std["acc_vector"])), abs(max(df_transformed["acc_vector"]) - (mean["acc_vector"]+k*std["acc_vector"]))]
    z_acc = max(range_acc)  
    q_acc = np.random.uniform(0,z_acc)
    
    range_gyro = [abs(min(df_transformed["gyro_vector"]) - (mean["gyro_vector"]-k*std["gyro_vector"])), abs(max(df_transformed["gyro_vector"]) - (mean["gyro_vector"]+k*std["gyro_vector"]))]
    z_gyro = max(range_gyro)  
    q_gyro = np.random.uniform(0,z_gyro)
    
    range_mag = [abs(min(df_transformed["mag_vector"]) - (mean["mag_vector"]-k*std["mag_vector"])), abs(max(df_transformed["mag_vector"]) - (mean["mag_vector"]+k*std["mag_vector"]))]
    z_mag = max(range_mag)  
    q_mag = np.random.uniform(0,z_mag)
        
    if (density_acc < x):
        index_outliers = np.random.choice(df_transformed.index[~df_transformed['outlier_acc']].tolist(), size=int(np.ceil((x-density_acc)/100*len(df_transformed))), replace = False)
        index_outliers = np.append(index_outliers, df_transformed.index[df_transformed['outlier_acc']].tolist())
    else:
        index_outliers = df_transformed.index[df_transformed['outlier_acc']].tolist()
    df_transformed["acc_injection"] = df_transformed["acc_vector"]    
    df_transformed.loc[index_outliers, "acc_injection"] = mean[0] + s * k * (std[0]+q_acc)
    
    if (density_gyro < x):  
        index_outliers = np.random.choice(df_transformed.index[~df_transformed['outlier_gyro']].tolist(), size=int(np.ceil((x-density_gyro)/100*len(df_transformed))), replace = False)
        index_outliers = np.append(index_outliers, df_transformed.index[df_transformed['outlier_gyro']].tolist())
    else:
        index_outliers = df_transformed.index[df_transformed['outlier_gyro']].tolist()
        
    df_transformed["gyro_injection"] = df_transformed["gyro_vector"]    
    df_transformed.loc[index_outliers, "gyro_injection"] = mean[1] + s * k * (std[1]+q_gyro)
    
    if (density_mag < x):
        index_outliers = np.random.choice(df_transformed.index[~df_transformed['outlier_mag']].tolist(), size=int(np.ceil((x-density_mag)/100*len(df_transformed))), replace = False)
        index_outliers = np.append(index_outliers, df_transformed.index[df_transformed['outlier_mag']].tolist())
    else:
        index_outliers = df_transformed.index[df_transformed['outlier_mag']].tolist()
    df_transformed["mag_injection"] = df_transformed["mag_vector"]    
    df_transformed.loc[index_outliers, "mag_injection"] = mean[2] + s * k * (std[2]+q_acc)
    
    # gyro_injection existe um grande valor minimo devido à amplitude maxima relativamente ao outlier
    # o valor médio e máximo estão muito afastados
    return df_transformed[["acc_injection","gyro_injection","mag_injection"]].describe().loc[["min","max","mean","std","count"],["acc_injection","gyro_injection","mag_injection"]]

In [20]:
# exercise 3.9 using single value decomposition

#def regression(x,Y):
#    n, p = np.shape(x)
#    X = np.append(np.ones((n,1)),x,axis=1)
#    U,s,V = linalg.svd(X)
#    S = np.zeros((p+1,n))
#    for i in range(0,p):
#        S[i,i] = 1/(s[i])
#    pseudo_inverse = np.dot(V, np.dot(S,np.transpose(U)))
#    beta = np.dot(pseudo_inverse,Y)
    
#    return beta

In [21]:
# exercise 3.9

def regression(x,Y):
    n = len(x)
    z = np.ones(n)
    X = np.transpose(np.vstack([z,x]))

    pseudo_inverse = np.dot(linalg.inv((np.dot(np.transpose(X),X))), np.transpose(X))
    # pseudo_inverse = linalg.pinv(X)
    beta = np.dot(pseudo_inverse,Y)
    return beta

In [22]:
# exercise 3.9

# https://towardsdatascience.com/linear-regression-from-scratch-cd0dee067f72

def plot_acc_regression(act,k):

    df_transformed = transformed_dataset()
    df_transformed = df_transformed[df_transformed["activity"]==act].reset_index(drop=True)
    mean = df_transformed[["acc_vector","gyro_vector","mag_vector"]].mean()
    std = df_transformed[["acc_vector","gyro_vector","mag_vector"]].std()
    
    Z = (df_transformed[["acc_vector","gyro_vector","mag_vector"]]-mean)/std
    outliers = abs(Z)>k
    non_acc_outliers = df_transformed[outliers["acc_vector"]==False][["acc_vector", "time"]]
    
    beta=regression(df_transformed["time"],df_transformed["acc_vector"])

    df_non_outliers = transformed_dataset()
    df_non_outliers = df_transformed[df_transformed["activity"]==act]

    beta_non_outliers=regression(non_acc_outliers["time"],non_acc_outliers["acc_vector"])

    fig, ax = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))

    X = df_transformed["time"]
    Y = df_transformed["acc_vector"]
    x_max = np.max(X) + 50
    x_min = np.min(X) - 50

    x = np.linspace(x_min, x_max)
    y = beta[0]+beta[1]*x
    ax[0].plot(x, y, color='b', label='Linear Regression')
    ax[0].scatter(X, Y, color='g', label='Data Point')
    ax[0].set_title("acc_vector over time for activity 1", size=14)
    ax[0].set_xlabel("time", size=14)
    ax[0].set_ylabel("acc_vector", size=14)
    ax[0].legend(prop=dict(size=12))

    X = non_acc_outliers["time"]
    Y = non_acc_outliers["acc_vector"]
    x_max = np.max(X) + 50
    x_min = np.min(X) - 50

    x = np.linspace(x_min, x_max)
    y = beta_non_outliers[0]+beta_non_outliers[1]*x
    ax[1].plot(x, y, color='b', label='Linear Regression')
    ax[1].scatter(X, Y, color='g', label='Data Point')
    ax[1].set_title("acc_vector without outliers over time for activity 1", size=14)
    ax[1].set_xlabel("time", size=14)
    ax[1].set_ylabel("acc_vector", size=14)
    ax[1].legend(prop=dict(size=12), loc=1)
    
    plt.savefig("acc_vector over time for activity 1")
    plt.show()

In [23]:
# exercise 3.10
def linear_injection_anterior_window(n,k,act):    
    density_acc, density_gyro, density_mag, df_activity = density_activity(k,act)
    df_activity = df_activity.sort_values(by=["time"])
    df_activity = df_activity.reset_index(drop=True)
    window = np.zeros((len(df_activity)-n+1,4))
    for row in range(0, len(df_activity)-n):
        x = df_activity.loc[np.arange(row,row+n),"time"].reset_index(drop=True)
        Y = df_activity.loc[np.arange(row,row+n),"acc_vector"]
        beta = regression(x,Y)
        window[row,:] = [row, row+n, beta[0],beta[1]]
        
    mean = df_activity[["acc_vector"]].mean()
    std = df_activity[["acc_vector"]].std()
    
    x_density =10
    if (density_acc < x_density):
        index_outliers = np.random.choice(df_activity.index[~df_activity['outlier_acc']].tolist(), size=int(np.ceil((x_density-density_acc)/100*len(df_activity))), replace = False)
        index_outliers = np.append(index_outliers, df_activity.index[df_activity['outlier_acc']].tolist())
    else:
        index_outliers = df_activity.index[df_activity['outlier_acc']].tolist()
    index_outliers.sort()
    
    
    x_outliers = df_activity.loc[index_outliers,"time"].reset_index(drop=True)
    
    error_injection = np.zeros(len(x_outliers))
    Y_outlier = np.zeros(len(x_outliers))
    for i in range(0,len(index_outliers)):
        time_outlier = x_outliers[i]
        position_outlier = index_outliers[i]
        window_number = position_outlier==window[:,1]
        if sum(window_number) == 0:
            w = 0
        else:
            w = np.argwhere(window_number==True)
        Y_outlier[i] = np.dot([1,time_outlier],np.transpose(window[w,[2,3]]))
        error_injection[i] = abs(df_activity.loc[index_outliers[i],"acc_vector"] - Y_outlier[i])    
                
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 6))
    ax[0].scatter(x_outliers, Y_outlier, color = 'red', zorder=20, label='predicted value')
    ax[1].scatter(x_outliers, df_activity.loc[index_outliers,"acc_vector"], color = 'blue', zorder=0, alpha=0.6, s=0.3, label='real value')
    ax[0].set_title("Predicted values using linear regression for activity {}".format(act), size=15)
    ax[0].set_xlabel("Time", size=15)
    ax[0].set_ylabel("Predicted Acc_vector", size=15)
    ax[1].set_title("Real values for activity {}".format(act), size=15)
    ax[1].set_xlabel("Time", size=15)
    ax[1].set_ylabel("Acc_vector", size=15)
    ax[2].hist(error_injection,bins=40)
    ax[2].set_title("Histogram of injection error for activity {}".format(act))
    plt.savefig("Injection_error_activity{}".format(act))

In [24]:
# exercise 3.10
def linear_injection_cross_validation(n,parts,act,k):
    rmse = np.zeros((parts-1,1))
    density_acc, density_gyro, density_mag, df_activity = density_activity(k,act)
    df_activity = df_activity.sort_values(by=["time"])
    df_activity = df_activity.reset_index(drop=True)
    size = int(np.ceil(len(df_activity)/parts))
    for k1 in range(1,parts):
        time = df_activity.loc[np.arange(size*k1,min(size*(k1+1),len(df_activity)-1)),"time"].reset_index(drop=True)
        acc_real = df_activity.loc[np.arange(size*k1,min(size*(k1+1),len(df_activity)-1)),"acc_vector"].reset_index(drop=True)
        acc_test = np.zeros(len(time))
        x = df_activity.loc[np.arange(size*k1-n,size*k1),"time"].reset_index(drop=True)
        Y = df_activity.loc[np.arange(size*k1-n,size*k1),"acc_vector"]
        beta = regression(x,Y)
        for j in range(0,len(time)):
            acc_test[j] = np.dot([1,time[j]],np.transpose(beta))
            x = np.append(x[1:], time[j])
            Y = np.append(Y[1:], acc_test[j])
            beta = regression(x,Y)
        error = abs(acc_real-acc_test)
        rmse[k1-1,0] = np.sqrt(sum(error**2)/size)
    rmse_mean = rmse.mean()
    
    return rmse_mean

In [25]:
# exercise 3.10
def best_n(act,k):
    
    df_transformed = transformed_dataset()
    
    number = int(np.floor(len(df_transformed[df_transformed["activity"]==act].copy())/32)*2)
    n = 2*number + number *np.arange(0,3)
    rmse_mean = np.zeros((len(n),1))
    for j in range(0,len(n)):
        rmse_mean[j] = linear_injection_cross_validation(n[j],4,act,k)
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
    ax.plot(n,rmse_mean)
    ax.set_xlabel("n", size=15)
    ax.set_ylabel("RMSE mean", size=15)
    ax.set_title("RMSE mean for n for anterior regression for activity {}".format(act), size=15)
    plt.savefig("RMSE_mean_anterior_activity{}".format(act))
    best = n[np.argmin(rmse_mean)]
    linear_injection_anterior_window(best,k,act)
    
    return best

In [26]:
# exercise 3.11
def linear_injection_centered_window(n,k,act):    
    density_acc, density_gyro, density_mag, df_activity = density_activity(k,act)
    df_activity = df_activity.sort_values(by=["time"])
    df_activity = df_activity.reset_index(drop=True)
    window = np.zeros((len(df_activity)-n+1,3))
    for row in range(int(n/2), int(len(df_activity)-n/2)):
        positions = [i for i in np.arange(row-n/2,row+n/2) if i not in [row]]
        x = df_activity.loc[positions,"time"].reset_index(drop=True)
        Y = df_activity.loc[positions,"acc_vector"].reset_index(drop=True)
        beta = regression(x,Y)
        window[row,:] = [row, beta[0],beta[1]]
         
    mean = df_activity[["acc_vector"]].mean()
    std = df_activity[["acc_vector"]].std()
    
    x_density =10
    if (density_acc < x_density):
        index_outliers = np.random.choice(df_activity.index[~df_activity['outlier_acc']].tolist(), size=int(np.ceil((x_density-density_acc)/100*len(df_activity))), replace = False)
        index_outliers = np.append(index_outliers, df_activity.index[df_activity['outlier_acc']].tolist())
    else:
        index_outliers = df_activity.index[df_activity['outlier_acc']].tolist()
    index_outliers.sort()
    
    
    x_outliers = df_activity.loc[index_outliers,"time"].reset_index(drop=True)
    
    error_injection = np.zeros(len(x_outliers))
    Y_outlier = np.zeros(len(x_outliers))
    for i in range(0,len(index_outliers)):
        time_outlier = x_outliers[i]
        position_outlier = index_outliers[i]
        window_number = position_outlier==window[:,0]
        if sum(window_number) == 0:
            if position_outlier < window[0,0]:
                w = 0
            else:
                w = len(window)-1  
        else:
            w = np.argwhere(window_number==True)
        Y_outlier[i] = np.dot([1,time_outlier],np.transpose(window[w,[1,2]]))
        error_injection[i] = abs(df_activity.loc[index_outliers[i],"acc_vector"] - Y_outlier[i])    
             
    fig, ax = plt.subplots(nrows=1, ncols=3, figsize=(15, 6))
    ax[0].scatter(x_outliers, Y_outlier, color = 'red', zorder=20, label='predicted value')
    ax[1].scatter(x_outliers, df_activity.loc[index_outliers,"acc_vector"], color = 'blue', zorder=0, alpha=0.6, s=0.3, label='real value')
    ax[0].set_title("Predicted values using linear regression for activity {}".format(act), size=15)
    ax[0].set_xlabel("Time", size=15)
    ax[0].set_ylabel("Predicted Acc_vector", size=15)
    ax[1].set_title("Real values for activity {}".format(act), size=15)
    ax[1].set_xlabel("Time", size=15)
    ax[1].set_ylabel("Acc_vector", size=15)
    ax[2].hist(error_injection,bins=40)
    ax[2].set_title("Histogram of injection error for activity {}".format(act))
    plt.savefig("Injection_error_activity{} for centered window".format(act))

In [27]:
# exercise 3.11
def linear_injection_cross_validation_centered(n,parts,act,k):
    rmse = np.zeros((parts-2,1))
    density_acc, density_gyro, density_mag, df_activity = density_activity(k,act)
    df_activity = df_activity.sort_values(by=["time"])
    df_activity = df_activity.reset_index(drop=True)
    size = int(np.ceil(len(df_activity)/parts))
    for k1 in range(1,parts-1):
        time = df_activity.loc[np.arange(size*k1,min(size*(k1+1),len(df_activity)-1)),"time"].reset_index(drop=True)
        acc_real = df_activity.loc[np.arange(size*k1,min(size*(k1+1),len(df_activity)-1)),"acc_vector"].reset_index(drop=True)
        acc_test = np.zeros(len(time))
        x = df_activity.loc[np.arange(size*k1-n/2,size*k1+n/2),"time"].reset_index(drop=True)
        Y = df_activity.loc[np.arange(size*k1-n/2,size*k1+n/2),"acc_vector"].reset_index(drop=True)
        beta = regression(x,Y)
        for j in range(0,len(time)):
            acc_test[j] = np.dot([1,time[j]],np.transpose(beta))
            x = np.append(x[1:], time[j])
            Y = np.append(Y[1:], acc_test[j])
            beta = regression(x,Y)
        error = abs(acc_real-acc_test)
        rmse[k1-1,0] = np.sqrt(sum(error**2)/size)
    rmse_mean = rmse.mean()
    
    return rmse_mean

In [28]:
# exercise 3.11
def best_n_centered(act,k):
    
    df_transformed = transformed_dataset()
    
    number = int(np.floor(len(df_transformed[df_transformed["activity"]==act].copy())/32)*2)
    n = 2*number + number *np.arange(0,3)
    rmse_mean = np.zeros((len(n),1))
    for j in range(0,len(n)):
        rmse_mean[j] = linear_injection_cross_validation_centered(n[j],4,act,k)
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(6, 6))
    ax.plot(n,rmse_mean)
    ax.set_xlabel("n", size=15)
    ax.set_ylabel("RMSE mean", size=15)
    ax.set_title("RMSE mean for n for centered regression for activity {}".format(act), size=15)
    plt.savefig("RMSE_mean_centered_activity{}".format(act))
    best = n[np.argmin(rmse_mean)]
    linear_injection_centered_window(best,k,act)
    
    return best

In [77]:
#exercice 4.1
def compare_means(): 
    
    warnings.filterwarnings("ignore")
    
    df_transformed = transformed_dataset()[["acc_vector","gyro_vector","mag_vector","activity"]]
    df_transformed["activity"] = df_transformed["activity"].astype(str)
    
    means = df_transformed.groupby('activity')['acc_vector','gyro_vector','mag_vector'].mean()
    means = means.reset_index()
    
    vector_names = ['acc_vector','gyro_vector','mag_vector']
    means_p = pd.DataFrame()
    means_p['activity'] = np.arange(1,17)
    norm_p = pd.DataFrame()
    norm_p['activity'] = np.arange(1,17)
    for v in range(0,3):
        vector = vector_names[v]
        #means_p = np.zeros((2,16))
        for act in range(1,17):
            data = df_transformed[df_transformed["activity"]==str(act)][vector]
            mean = means[means['activity']==str(act)][vector]
            stat, p = stats.ttest_1samp(data, popmean=mean)
            means_p.loc[act-1,vector+'_stat'] = stat.values[0]
            means_p.loc[act-1,vector+'_p'] = p[0]
        
        values_per_group = [col for col_name, col in df_transformed.groupby("activity")[vector]]
        stat_var, p_var = levene(*values_per_group)
        #stat_p = np.zeros((2,16))
        for act in range(1,17):
            data = df_transformed[df_transformed["activity"]==str(act)][vector]
            stat, p = shapiro(data)
            norm_p.loc[act-1,vector+'_stat'] = stat
            norm_p.loc[act-1,vector+'_p'] = p
        
        name = vector + ' ~ activity'
        lm = ols(name, data=df_transformed).fit()
        table = sm.stats.anova_lm(lm)
        print(table)
        comp = multi.MultiComparison(df_transformed[vector], df_transformed["activity"])
        tukey = comp.tukeyhsd()
        print(tukey)
        
        return means_p, norm_p, stat_var, p_var


In [30]:
# exercice 4.3 and 4.4
def pca_func(X):
     
    Zscore_scaler = StandardScaler()
    Zscore_scaler.fit(X)
    X_scaled=pd.DataFrame(Zscore_scaler.transform(X))
    
    pca = PCA(n_components=0.75)
    pca.fit(X_scaled)
    
    variance = pd.DataFrame(pca.explained_variance_ratio_*100, columns=["variance"])
    components = pd.DataFrame(pca.components_)
    
    sns.set_style("darkgrid")
    fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(10, 3))

    ax = sns.barplot(x=variance.index, y="variance", data=variance, color="blue")

    ax.set_ylabel('Percentage of Explained Variance (%)', size=12)
    ax.set_xlabel('Principal Components', size=12)
    ax.set_title("PCA", size=12)
    plt.savefig("PCA")

    return variance, components

In [31]:
# exercice 4.4.1

def pca_transformed(X):
    
    Zscore_scaler = StandardScaler()
    Zscore_scaler.fit(X)
    X_scaled=pd.DataFrame(Zscore_scaler.transform(X))
    
    pca = PCA(n_components=0.75)
    pca.fit(X_scaled)
    
    components = pd.DataFrame(pca.components_)
    
    X_pca = np.dot(components,np.transpose(X_scaled))
    X_pca = pd.DataFrame(X_pca).transpose()
    
    return X_pca

In [32]:
# exercice 4.4.1

# https://python-graph-gallery.com/372-3d-pca-result/

def pca_3d_plot():
    warnings.filterwarnings("ignore")

    df_transformed = transformed_dataset()
    df = df_transformed[["acc_vector","gyro_vector","mag_vector","activity"]]
    df['activity']=pd.Categorical(df['activity'])

    my_color=df['activity'].cat.codes
    df = df.drop('activity', 1)
    
    Zscore_scaler = StandardScaler()
    Zscore_scaler.fit(df)
    df=pd.DataFrame(Zscore_scaler.transform(df))
 
    # Run The PCA
    pca = PCA(n_components=3)
    pca.fit(df)
 
    # Store results of PCA in a data frame
    result=pd.DataFrame(pca.transform(df), columns=['PCA%i' % i for i in range(3)], index=df.index)

    # Plot initialisation
    fig = plt.figure(figsize=(10,10))
    ax = fig.add_subplot(111, projection='3d')
    a = ax.scatter(result['PCA0'], result['PCA1'], result['PCA2'], c=my_color, cmap="rainbow", s=60, alpha=0.5)

    # make simple, bare axis lines through space:
    xAxisLine = ((min(result['PCA0']), max(result['PCA0'])), (0, 0), (0,0))
    ax.plot(xAxisLine[0], xAxisLine[1], xAxisLine[2], 'r')
    yAxisLine = ((0, 0), (min(result['PCA1']), max(result['PCA1'])), (0,0))
    ax.plot(yAxisLine[0], yAxisLine[1], yAxisLine[2], 'r')
    zAxisLine = ((0, 0), (0,0), (min(result['PCA2']), max(result['PCA2'])))
    ax.plot(zAxisLine[0], zAxisLine[1], zAxisLine[2], 'r')
 
    # label the axes
    ax.set_xlabel("PC1", size=12)
    ax.set_ylabel("PC2", size=12)
    ax.set_zlabel("PC3", size=12)
    ax.set_title("Normalized principal directions", size=14)
    plt.savefig("Normalized principal directions")

    plt.show()

In [130]:
# exercice 4.4.1

#https://towardsdatascience.com/pca-using-python-scikit-learn-e653f8989e60

def pca_2d_plot(X_pca):

    fig = plt.figure(figsize = (8,8))
    ax = fig.add_subplot(1,1,1) 
    ax.set_xlabel('Principal Component 1', fontsize = 15)
    ax.set_ylabel('Principal Component 2', fontsize = 15)
    ax.set_title('2 component PCA', fontsize = 20)
    targets = [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16]
    colors = ["r","g","b","c","m","y","lime","indigo","black","grey","sienna","pink","deeppink","darkmagenta","blueviolet","darkorange"]
    for target, color in zip(targets,colors):
        indicesToKeep = X_pca['activity'] == target
        ax.scatter(X_pca.loc[indicesToKeep, 0], X_pca.loc[indicesToKeep, 1], s = 50)
    ax.legend(targets)
    ax.grid()

    plt.savefig("pca_2_components")
    plt.show()

In [33]:
# exercice 4.2

def dataset_act_vector(act,vector):    
    
    df = pd.DataFrame()
    vector_x = vector+'_x'
    vector_y = vector+'_y'
    vector_z = vector+'_z'

    for i in range (0,15):
        df = df.append(dataset(i))
    df = df.reset_index()
    df_act_vector = df[df["activity"]==act][[vector_x,vector_y,vector_z,'time']].reset_index(drop=True)
    df_act_vector = df_act_vector.sort_values(by=["time"]).reset_index(drop=True)
    
    return df_act_vector

In [34]:
# exercice 4.2

def statistical_features(initial,final,act,vector):
    
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    results = df_window.describe()
    variance = np.sqrt(results.loc['std']) 
    results.loc['variance'] = variance
    root_mean_square = np.sqrt(np.sum(df_window**2)/df_window.count())
    results.loc['rms'] = root_mean_square
    vector_x = vector+'_x'
    vector_y = vector+'_y'
    vector_z = vector+'_z'
    derivative = np.zeros((4))
    derivative[0] = np.mean(np.gradient(df_window[vector_x],df_window['time']))
    derivative[1] = np.mean(np.gradient(df_window[vector_y],df_window['time']))
    derivative[2] = np.mean(np.gradient(df_window[vector_z],df_window['time']))
    results.loc['av_der'] = derivative
    skewness = np.zeros(4)
    skewness[0] = skew(df_window[vector_x])
    skewness[1] = skew(df_window[vector_y])
    skewness[2] = skew(df_window[vector_z])
    results.loc['skew'] = skewness
    kurt = np.zeros(4)
    kurt[0] = kurtosis(df_window[vector_x])
    kurt[1] = kurtosis(df_window[vector_y])
    kurt[2] = kurtosis(df_window[vector_z])
    results.loc['kurtosis'] = kurt
    interq_range = results.loc['75%']-results.loc['25%']
    results.loc['interq_range'] = interq_range
    zero_crossings = np.zeros(4)
    #https://stackoverflow.com/questions/3843017/efficiently-detect-sign-changes-in-python
    zero_crossings[0] = len(np.where(np.diff(np.sign(df_window[vector_x])))[0])/(final-initial)
    zero_crossings[1] = len(np.where(np.diff(np.sign(df_window[vector_y])))[0])/(final-initial)
    zero_crossings[2] = len(np.where(np.diff(np.sign(df_window[vector_z])))[0])/(final-initial)
    results.loc['zero_crossings'] = zero_crossings
    mean_crossings = np.zeros(4)
    #https://stackoverflow.com/questions/3843017/efficiently-detect-sign-changes-in-python
    mean_crossings[0] = len(np.where(np.diff(np.sign(df_window[vector_x]-results.loc['mean'][vector_x])))[0])/(final-initial)
    mean_crossings[1] = len(np.where(np.diff(np.sign(df_window[vector_y]-results.loc['mean'][vector_y])))[0])/(final-initial)
    mean_crossings[2] = len(np.where(np.diff(np.sign(df_window[vector_z]-results.loc['mean'][vector_z])))[0])/(final-initial)
    results.loc['mean_crossings'] = mean_crossings
    pair_correlation = df_window.corr()
    results.loc['corr_'+vector_x] = pair_correlation.loc[vector_x]
    results.loc['corr_'+vector_y] = pair_correlation.loc[vector_y]
    results.loc['corr_'+vector_z] = pair_correlation.loc[vector_z]
    spec_entropy = np.zeros(4)
    p_data = df_window[vector_x].value_counts()
    spec_entropy[0] = entropy(p_data)
    p_data = df_window[vector_y].value_counts()
    spec_entropy[1] = entropy(p_data)
    p_data = df_window[vector_z].value_counts()
    spec_entropy[2] = entropy(p_data)
    results.loc['spec_entropy'] = spec_entropy
    
    return results

In [35]:
# exercice 4.2

def moviment_intensity(initial,final,act): 
    vector="acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = "acc_x"
    vector_y = "acc_y"
    vector_z = "acc_z"
    df_window = df_window[[vector_x,vector_y,vector_z]]
    MI = np.sqrt(np.sum(df_window**2,axis=1))
    AI = np.sum(MI)/(final-initial)
    VI = np.sum((MI-AI)**2)/(final-initial)
    
    return AI,VI

In [36]:
# exercice 4.2

def normalized_signal_magnitude_area(initial,final,act):
    vector="acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = "acc_x"
    vector_y = "acc_y"
    vector_z = "acc_z"
    df_window = df_window[[vector_x,vector_y,vector_z]]
    SMA = np.sum(np.sum(abs(df_window)))/(final-initial)
    
    return SMA

In [37]:
# exercice 4.2

def eigen_values_dominant_directions(initial,final,act):
    vector="acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = "acc_x"
    vector_y = "acc_y"
    vector_z = "acc_z"
    df_window = df_window[[vector_x,vector_y,vector_z]]
    covariance_matrix = np.cov(df_window)
    eigvals,eigvecs = eig(covariance_matrix)
    eigvals = np.array(sorted(eigvals, reverse=True)[:2])
    
    return eigvals

In [38]:
# exercice 4.2

def correlation_acceleration_gravity_heading_directions(initial,final,act):
    vector="acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = "acc_x"
    vector_y = "acc_y"
    vector_z = "acc_z"
    df_window = df_window[[vector_x,vector_y,vector_z]]
    gravity = df_window[vector_x]
    heading = df_window[[vector_y,vector_z]]
    heading_norm = np.sqrt(np.sum(heading**2,axis=1))
    df_gravity_heading = pd.DataFrame({'gravity':gravity,'heading_norm':heading_norm})
    correlation = df_gravity_heading.corr()
    correlation = correlation["heading_norm"][0]
    
    return correlation

In [39]:
# exercice 4.2

def averaged_velocity_heading_direction(initial,final,act):
    vector="acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_y = vector+'_y'
    vector_z = vector+'_z'
    velocity_y = np.sum(cumtrapz(df_window[vector_y],df_window['time']))/(final-initial)
    velocity_z = np.sum(cumtrapz(df_window[vector_z],df_window['time']))/(final-initial)
    norm_velocity = np.sqrt(velocity_y**2 + velocity_z**2)
    
    return norm_velocity

In [40]:
# exercice 4.2

def averaged_velocity_gravity_direction(initial,final,act):
    vector = "acc"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = vector+'_x'
    velocity_x = np.sum(cumtrapz(df_window[vector_x],df_window['time']))/(final-initial)
    
    return velocity_x

In [41]:
# exercice 4.2

def averaged_rotation_angles_gravity_direction(initial,final,act):
    vector="gyro"
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = vector+'_x'
    av_rotation = np.sum(df_window[vector_x])/(final-initial)
    
    return av_rotation

In [42]:
# exercice 4.2

def dominant_frequency(initial,final,act,vector):
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = vector+'_x'
    vector_y = vector+'_y'
    vector_z = vector+'_z'
    df_window = df_window[[vector_x,vector_y,vector_z]]
    fourier = fft.fft(df_window)
    freq = fft.fftfreq(len(fourier))
    index_max = np.unravel_index(abs(fourier).argmax(), fourier.shape)
    frequency = freq(index_max[0])
    
    return frequency

In [43]:
# exercice 4.2

def energy(initial,final,act,vector):
    df_act_vector = dataset_act_vector(act,vector)
    df_window = df_act_vector[(df_act_vector['time']>=initial) & (df_act_vector['time']<=final)].reset_index(drop=True)
    vector_x = vector+'_x'
    vector_y = vector+'_y'
    vector_z = vector+'_z'
    df_window = df_window[[vector_x,vector_y,vector_z]]
    dc = df_window - df_window.mean()
    fourier = np.fft.fft(dc)
    en_x,en_y,en_z = abs(fourier).sum(axis=0)/(final-initial)
    
    return en_x,en_y,en_z

In [44]:
# exercice 4.2

def averaged_acceleration_energy(initial,final,act):
    vector = 'acc'
    en_x,en_y,en_z = energy(initial,final,act,vector)
    aae = np.mean([en_x,en_y,en_z])
    
    return aae

In [45]:
# exercice 4.2

def averaged_rotation_energy(initial,final,act):
    vector = 'gyro'
    en_x,en_y,en_z = energy(initial,final,act,vector)
    are = np.mean([en_x,en_y,en_z])
    
    return are

In [46]:
# exercice 4.2

#window_size=40000
#act=1
#vector="acc"

def features_dataset_by_activity(window_size,act,vector):
    warnings.filterwarnings("ignore")
    
    data = pd.DataFrame(columns=[vector+"_x_mean",vector+"_y_mean",vector+"_z_mean",
                                 vector+"_x_median",vector+"_y_median",vector+"_z_median",
                                 vector+"_x_std",vector+"_y_std",vector+"_z_std",
                                 vector+"_x_variance",vector+"_y_variance",vector+"_z_variance",
                                 vector+"_x_rms",vector+"_y_rms",vector+"_z_rms",
                               #  vector+"_x_av_der",vector+"_y_av_der",vector+"_z_av_der",
                                 vector+"_x_skew",vector+"_y_skew",vector+"_z_skew",
                                 vector+"_x_kurtosis",vector+"_y_kurtosis",vector+"_z_kurtosis",
                                 vector+"_x_interq_range",vector+"_y_interq_range",vector+"_z_interq_range",
                                 vector+"_x_zero_crossings",vector+"_y_zero_crossings",vector+"_z_zero_crossings",
                                 vector+"_x_mean_crossings",vector+"_y_mean_crossings",vector+"_z_mean_crossings",
                                 vector+"_x_corr_acc_x",vector+"_y_corr_acc_x",vector+"_z_corr_acc_x",
                                 vector+"_x_corr_acc_y",vector+"_y_corr_acc_y",vector+"_z_corr_acc_y",
                                 vector+"_x_corr_acc_z",vector+"_y_corr_acc_z",vector+"_z_corr_acc_z",
                                 vector+"_x_spec_entropy",vector+"_y_spec_entropy",vector+"_z_spec_entropy",
                                 
                                 "acc_MI","acc_VI","acc_SMA","acc_gravity_heading_corr","acc_avg_vel_head", "acc_avg_vel_grav", 
                                 "gyro_avg_rot",vector+"_x_en",vector+"_y_en",vector+"_z_en","acc_aae","gyro_are"])

    df_win=pd.DataFrame(columns=["init","final"])

    df = dataset_act_vector(act,vector)

    for i in range(0,len(df)-window_size,int(window_size/2)):
        df_win.loc[i]=[df["time"][i],df["time"][i+window_size]]
    
    df_win=df_win.reset_index(drop=True)
    
    for i in range(0,len(df_win)):
        initial = df_win["init"][i]
        final = df_win["final"][i]
    
        results = statistical_features(initial,final,act,vector)
        ai,vi = moviment_intensity(initial,final,act)
        sma = normalized_signal_magnitude_area(initial,final,act)
        #eigvals = eigen_values_dominant_directions(initial,final,act)
        corr = correlation_acceleration_gravity_heading_directions(initial,final,act)
        avg_vel_head = averaged_velocity_heading_direction(initial,final,act)
        avg_vel_grav = averaged_velocity_gravity_direction(initial,final,act)
        avg_rot = averaged_rotation_angles_gravity_direction(initial,final,act)
        en_x,en_y,en_z = energy(initial,final,act,vector)
        aae = averaged_acceleration_energy(initial,final,act)
        are = averaged_rotation_energy(initial,final,act)
    
        data.loc[i]=[results.loc["mean"][0],results.loc["mean"][1],results.loc["mean"][2],
                     results.loc["50%"][0],results.loc["50%"][1],results.loc["50%"][2],
                     results.loc["std"][0],results.loc["std"][1],results.loc["std"][2],
                     results.loc["variance"][0],results.loc["variance"][1],results.loc["variance"][2],
                     results.loc["rms"][0],results.loc["rms"][1],results.loc["rms"][2],
                   #  results.loc["av_der"][0],results.loc["av_der"][1],results.loc["av_der"][2],
                     results.loc["skew"][0],results.loc["skew"][1],results.loc["skew"][2],
                     results.loc["kurtosis"][0],results.loc["kurtosis"][1],results.loc["kurtosis"][2],
                     results.loc["interq_range"][0],results.loc["interq_range"][1],results.loc["interq_range"][2],
                     results.loc["zero_crossings"][0],results.loc["zero_crossings"][1],results.loc["zero_crossings"][2],
                     results.loc["mean_crossings"][0],results.loc["mean_crossings"][1],results.loc["mean_crossings"][2],
                     results.loc["corr_acc_x"][0],results.loc["corr_acc_x"][1],results.loc["corr_acc_x"][2],
                     results.loc["corr_acc_y"][0],results.loc["corr_acc_y"][1],results.loc["corr_acc_y"][2],
                     results.loc["corr_acc_z"][0],results.loc["corr_acc_z"][1],results.loc["corr_acc_z"][2],
                     results.loc["spec_entropy"][0],results.loc["spec_entropy"][1],results.loc["spec_entropy"][2],
                     
                     ai,vi,sma,corr,avg_vel_head,avg_vel_grav,avg_rot,en_x,en_y,en_z,aae,are]
        
    return data

In [87]:
# exercice 4.2

def feature_correlation_plot(x):
    
    plt.figure(figsize=(12,8))
    sns.heatmap(x)

    plt.title("Correlation between features", size=12) 
    plt.savefig("feature_dataset_correlation")

    plt.show()

In [83]:
# exercice 4.2

def features_dataset_activities(window_size,vector):

    data_final = []
    for i in range (1,17):
        data = features_dataset_by_activity(window_size,i,vector)
        data["activity"] = i
        data_final.append(data)
    features_data = pd.concat(data_final).reset_index(drop=True)
    
    return features_data

In [49]:
# exercice 4.2

def features_dataset_plot(features_data):

    fig, ax = plt.subplots(nrows=1, ncols=4, figsize=(15, 5))

    sns.scatterplot(x=features_data["acc_x_mean"], y=features_data["acc_y_mean"], ax=ax[0], hue=features_data["activity"],palette="bright" )
    ax[0].set_title("ACC_X_MEAN vs ACC_Y_MEAN", size=12)
    ax[0].set_xlabel("ACC_X_MEAN", size=12) 
    ax[0].set_ylabel("", size=12)
    ax[0].legend(prop=dict(size=7))

    sns.scatterplot(x=features_data["acc_x_std"], y=features_data["acc_z_std"], ax=ax[1], hue=features_data["activity"],palette="bright" )
    ax[1].set_title("ACC_X_STD vs ACC_Y_STD", size=12)
    ax[1].set_xlabel("ACC_X_STD", size=12) 
    ax[1].set_ylabel("", size=12)
    ax[1].legend(prop=dict(size=7))

    sns.scatterplot(x=features_data["acc_MI"], y=features_data["acc_SMA"], ax=ax[2], hue=features_data["activity"],palette="bright" )
    ax[2].set_title("AI vs SMA", size=12)
    ax[2].set_xlabel("AI", size=12) 
    ax[2].set_ylabel("", size=12)
    ax[2].legend(prop=dict(size=7))

    sns.scatterplot(x=features_data["acc_gravity_heading_corr"], y=features_data["gyro_avg_rot"], ax=ax[3], hue=features_data["activity"],palette="bright" )
    ax[3].set_title("CAGH vs ARATG", size=12)
    ax[3].set_xlabel("CAGH", size=12) 
    ax[3].set_ylabel("", size=12)
    ax[3].legend(prop=dict(size=7))

    plt.savefig("features_dataset")
    plt.plot()